In [1]:
# Econ 570 Big Data Project
# Group 17
# Chen-Lun Kao, Ruoyin Liu

import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk import punkt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import os
import collections
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from __future__ import division, print_function
from gensim import models
from keras.layers import Dense, Dropout,concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
import gensim.downloader as api
import wget
import gzip
# pip install tensorflow
# pip install wget

[nltk_data] Downloading package punkt to /Users/allan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/allan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/Users/allan/.conda/envs/fin/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
np.random.seed(500)

In [3]:
import pandas as pd
data = pd.read_csv('~/Desktop/17.csv',
                   header = None,
                   sep=',')
data.columns = ['Text', 'Label']
data.head()

,Text,Label
0,struggling depression anxiety amd mental healt...,1
1,boyfriend dumped saturday havent spoke longest...,1
2,im ready die im ready pass away longer feel tr...,1
3,hi neighbor lets diana newer friend showing si...,1
4,forget people dont feel things dont gut wrentc...,1


In [4]:
data.shape

(2000, 2)

In [5]:
data.Label.value_counts()

1    1000
0    1000
Name: Label, dtype: int64

In [6]:
pos = []
neg = []
for l in data.Label:
    if l == 0:
        pos.append(0)
        neg.append(1)
    elif l == 1:
        pos.append(1)
        neg.append(0)

In [7]:
data['Pos']= pos
data['Neg']= neg

In [8]:
data.head()

,Text,Label,Pos,Neg
0,struggling depression anxiety amd mental healt...,1,1,0
1,boyfriend dumped saturday havent spoke longest...,1,1,0
2,im ready die im ready pass away longer feel tr...,1,1,0
3,hi neighbor lets diana newer friend showing si...,1,1,0
4,forget people dont feel things dont gut wrentc...,1,1,0


In [9]:
def remove_punct(text):
    text = str(text)
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

data['Text_Clean'] = data['Text'].apply(lambda x: remove_punct(x))

In [10]:
from nltk import word_tokenize, WordNetLemmatizer
tokens = [word_tokenize(sen) for sen in data.Text_Clean]

In [11]:
def lower_token(tokens):
    return [w.lower() for w in tokens]

lower_tokens = [lower_token(token) for token in tokens]

In [12]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

In [13]:
def remove_stop_words(tokens):
    return [word for word in tokens if word not in stoplist]

In [14]:
filtered_words = [remove_stop_words(sen) for sen in lower_tokens]

In [15]:
result = [' '.join(sen) for sen in filtered_words]

In [16]:
data['Text_Final'] = result

In [17]:
data['tokens'] = filtered_words

In [18]:
data = data[['Text_Final', 'tokens', 'Label', 'Pos', 'Neg']]

In [19]:
data[:4]

,Text_Final,tokens,Label,Pos,Neg
0,struggling depression anxiety amd mental healt...,"[struggling, depression, anxiety, amd, mental,...",1,1,0
1,boyfriend dumped saturday havent spoke longest...,"[boyfriend, dumped, saturday, havent, spoke, l...",1,1,0
2,im ready die im ready pass away longer feel tr...,"[im, ready, die, im, ready, pass, away, longer...",1,1,0
3,hi neighbor lets diana newer friend showing si...,"[hi, neighbor, lets, diana, newer, friend, sho...",1,1,0


In [20]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(data['Text_Final'],data['Label'],test_size=0.3)

In [21]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [22]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(data['Text_Final'])

TfidfVectorizer(max_features=5000)

In [23]:
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [24]:
# This is the Naive Bayes model

In [25]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)

MultinomialNB()

In [26]:
predictions_NB = Naive.predict(Test_X_Tfidf)

In [27]:
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  89.83333333333333


In [28]:
#This is the SVM model

In [29]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)

SVC(gamma='auto', kernel='linear')

In [30]:
predictions_SVM = SVM.predict(Test_X_Tfidf)

In [31]:
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  91.16666666666666


In [32]:
#This is the CNN model

In [33]:
data_train, data_test = train_test_split(data, test_size=0.30)
test_y= data_test.Label

In [34]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

35776 words total, with a vocabulary size of 6537
Max sentence length is 925


In [35]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

19260 words total, with a vocabulary size of 4436
Max sentence length is 917


In [36]:
word2vec_path = '~/Desktop/GoogleNews-vectors-negative300.bin'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [37]:

def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors,
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [38]:

training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)

In [39]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

In [40]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 6537 unique tokens.


In [41]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [42]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(6538, 300)


In [43]:
test_sequences = tokenizer.texts_to_sequences(data_test["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [44]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):

    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)

    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [45]:
label_names = ['Pos', 'Neg']

In [46]:
y_train = data_train[label_names].values

In [47]:
x_train = train_cnn_data
y_tr = y_train

In [48]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM,
                len(list(label_names)))

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 50, 300)      1961400     ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 49, 200)      120200      ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 48, 200)      180200      ['embedding[0][0]']              
                                                                                              

2021-12-15 21:35:48.150956: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [49]:
num_epochs = 20
batch_size = 30

In [50]:
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

Epoch 1/20
42/42 [==============================] - 2s 39ms/step - loss: 0.3841 - acc: 0.8563 - val_loss: 0.3307 - val_acc: 0.8929

In [51]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

1/1 [==============================] - 0s 234ms/step


In [52]:
labels = [1, 0]

In [53]:
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

In [54]:
from sklearn.metrics import precision_score,f1_score,accuracy_score

In [55]:
precision = precision_score(test_y,prediction_labels)
print('Precision: %f' % precision)
f1 = f1_score(test_y,prediction_labels)
print('F1 score: %f' % f1)
accuracy = accuracy_score(test_y,prediction_labels)
print('Accuracy score: %f' % accuracy)

Precision: 0.955172
F1 score: 0.903752
Accuracy score: 0.901667


In [56]:
from sklearn.metrics import classification_report
print(classification_report(test_y,prediction_labels))


              precision    recall  f1-score   support

           0       0.85      0.95      0.90       277
           1       0.96      0.86      0.90       323

    accuracy                           0.90       600
   macro avg       0.90      0.91      0.90       600
weighted avg       0.91      0.90      0.90       600

